In [1]:
%%capture
!pip install transformers[sentencepiece]

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline
import pandas as pd

In [3]:
# loading aspect-based sentiment analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

#loading model from huggingface
sentiment_model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

...

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
def absa_sentiment(sentence : str, aspect1 : str, aspect2 : str):
  # print(f"Sentence : {sentence}")
  # print()
  labels = ["negative", "neutral", "positive"]

  inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect1} [SEP]",return_tensors="pt")
  outputs = absa_model(**inputs)
  probs = F.softmax(outputs.logits, dim=1).detach().numpy()[0]
  #print(f"Sentiment of aspect '{aspect1}' is : ")
  max_prob_index = probs.argmax()
  max_label = labels[max_prob_index]
  max_prob = probs[max_prob_index]
  # print(f"Label {max_label}: {max_prob:.4f}")
  # print()
  aspect1_sentiment = max_label

  inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect2} [SEP]", return_tensors="pt")
  outputs = absa_model(**inputs)
  probs = F.softmax(outputs.logits, dim=1)
  probs = probs.detach().numpy()[0]
  #print(f"Sentiment of aspect '{aspect2}' is : ")
  max_prob_index = probs.argmax()
  max_label = labels[max_prob_index]
  max_prob = probs[max_prob_index]
  #print(f"Label {max_label}: {max_prob:.4f}")
  #print()
  aspect2_sentiment = max_label

  sentiment = sentiment_model([sentence])[0]
  # print(f"Overall sentiment: {sentiment['label']} with score {sentiment['score']}")
  overall_sentiment = sentiment['label']

  return aspect1_sentiment, aspect2_sentiment, overall_sentiment

In [5]:
def capture_predicted_polarity(sentence: str, aspect: str):
    # We call absa_sentiment with the same aspect for both arguments
    # since we're only interested in one aspect at a time
    aspect1_sentiment, _, _ = absa_sentiment(sentence, aspect, aspect)
    return aspect1_sentiment

In [6]:
sentence = "The company's pricing strategy is commendable, offering excellent value for money. Their competitive rates and transparent fee structure make their products accessible to a wide range of customers without compromising on quality. However, the execution of their services leaves much to be desired. Delivery times are consistently longer than promised, customer support is frustratingly unresponsive, and the product often arrives in a condition that suggests careless handling during shipping. While the price point is attractive, the subpar execution significantly diminishes the overall customer experience, making one question whether the savings are worth the headache."

aspect1 = "pricing"
aspect2 = "execution"
absa_sentiment(sentence, aspect1, aspect2)

('positive', 'negative', 'positive')

In [7]:
restaraunt_df = pd.read_csv("/content/Laptop_Train_v2.csv", encoding = 'utf-8')

In [8]:
restaraunt_df2 = pd.read_csv("/content/Restaurants_Test_Data_PhaseB.csv", encoding = 'utf-8')

In [9]:
restaraunt_df

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12
...,...,...,...,...,...,...
2353,2272,We also use Paralles so we can run virtual mac...,Windows Server Enterprise 2003,neutral,104,134
2354,2272,We also use Paralles so we can run virtual mac...,Windows Server 2008 Enterprise,neutral,140,170
2355,848,"How Toshiba handles the repair seems to vary, ...",repair,conflict,24,30
2356,848,"How Toshiba handles the repair seems to vary, ...",repair,positive,130,136


In [21]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

def evaluate_sentiment_predictions(df, absa_sentiment_func):
    predicted_labels = []
    true_labels = restaraunt_df.polarity.tolist()

    for _, row in df.iterrows():
        sentence = row['Sentence']
        aspect = row['Aspect Term']
        #true_polarity = row['polarity']

        predicted_polarity = capture_predicted_polarity(sentence, aspect)

        # Handle 'conflict' label if present in your dataset
        #if true_polarity == 'conflict':
        #    continue  # Skip this sample or handle it as per your requirements

        #true_labels.append(true_polarity)
        predicted_labels.append(predicted_polarity)

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Generate a classification report
    report = classification_report(true_labels, predicted_labels)

    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(report)

# Usage
# df = pd.read_csv('your_dataframe.csv')  # Load your DataFrame
# evaluate_sentiment_predictions(df, absa_sentiment)

In [22]:
evaluate_sentiment_predictions(restaraunt_df, absa_sentiment)

Accuracy: 0.9449

Classification Report:
              precision    recall  f1-score   support

    conflict       0.00      0.00      0.00        45
    negative       0.95      0.98      0.96       866
     neutral       0.89      0.94      0.91       460
    positive       0.97      0.96      0.97       987

    accuracy                           0.94      2358
   macro avg       0.70      0.72      0.71      2358
weighted avg       0.93      0.94      0.94      2358



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
